In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import zipfile
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
from torchvision import transforms, models
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from tqdm import notebook
from skimage import io
from PIL import Image

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset, DataLoader, random_split
import torch.utils.model_zoo as model_zoo

#Kaggle Data importation

In [ ]:
train_data = pd.read_csv("../input/aerial-cactus-identification/train.csv", dtype=str)
test_data = pd.read_csv("../input/aerial-cactus-identification/sample_submission.csv", dtype=str)

FileNotFoundError: ignored

In [ ]:
zip_ref_train = zipfile.ZipFile("../input/aerial-cactus-identification/train.zip","r")
zip_ref_train.extractall()

In [ ]:
zip_ref_test = zipfile.ZipFile("../input/aerial-cactus-identification/test.zip","r")
zip_ref_test.extractall()

In [ ]:
train_data.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [ ]:
train_csv_path = "../input/aerial-cactus-identification/train.csv"
train_path = "../output/kaggle/working/train"
train_set = CactusDataset(train_csv_path, train_path)

trainloader = torch.utils.data.DataLoader(train_set,batch_size=batch_size,shuffle=False,num_workers=2)

test_csv_path = "../input/aerial-cactus-identification/sample_submission.csv"
test_path = "../output/kaggle/working/test"
test_set = CactusDataset(test_csv_path, test_path)

testloader = torch.utils.data.DataLoader(test_set,batch_size=batch_size,shuffle=False,num_workers=2)




FileNotFoundError: ignored

#Colab Data importation

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
path_to_my_drive = "/content/drive/MyDrive/Colab Notebooks/aerial-cactus-identification/"
os.chdir(path_to_my_drive)

train_data = pd.read_csv("train.csv", dtype=str)
test_data = pd.read_csv("sample_submission.csv", dtype=str)

In [6]:
train_data.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


# Classe Dataset et modèle

In [7]:
#Partie ajoutée lundi 17 mai
#La classe CactusDataset (qui hérite de Dataset) permet de créer un dataset a partir des fichiers 
#Pour plus d'explications voir : https://pytorch.org/tutorials/recipes/recipes/custom_dataset_transforms_loader.html

class CactusDataset(Dataset):
    """Cactus Dataset"""
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir 
        self.transform = transform
        self.labels = pd.read_csv(csv_file)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        #if self.labels[idx]:
         #   idx = idx.to_list()

        #os.path.join fabrique le chemin de l'image à partir de celui du dossier et du nom de l'image
        img_path = os.path.join(self.root_dir, self.labels.iloc[idx, 0]) 
        image = Image.open(img_path)
        
        label = self.labels.has_cactus[idx]

        
        #image = image.transpose(0, 2)
        if self.transform:
            image = self.transform(image)

            
        return [image, label]
        

In [8]:
batch_size = 64

In [9]:
net = models.vgg16(pretrained=True)
net.classifier[6] = nn.Linear(4096, 2)

#for param in net.features.parameters():
#    param.requires_grad = False



Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [10]:
torchvision.transforms.functional.resize
transform = transforms.Compose(
    [
     #transforms.Resize(size=(224, 224)), #modifier la taille
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,)),
])

In [11]:
train_csv_path = "train.csv"
train_path = "train"
train_set = CactusDataset(train_csv_path, train_path, transform)

test_csv_path = "sample_submission.csv"
test_path = "test"
test_set = CactusDataset(test_csv_path, test_path, transform)

trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=0)

# Inutile

In [ ]:
__all__ = [
    'VGG', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn',
    'vgg19_bn', 'vgg19',
]

model_urls = {
    'vgg11': 'https://download.pytorch.org/models/vgg11-bbd30ac9.pth',
    'vgg13': 'https://download.pytorch.org/models/vgg13-c768596a.pth',
    'vgg16': 'https://download.pytorch.org/models/vgg16-397923af.pth',
    'vgg19': 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth',
    'vgg11_bn': 'https://download.pytorch.org/models/vgg11_bn-6002323d.pth',
    'vgg13_bn': 'https://download.pytorch.org/models/vgg13_bn-abd245e5.pth',
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
    'vgg19_bn': 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth',
}


In [ ]:
class VGG(nn.Module):

    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


def make_layers(cfg, batch_norm=False):
    layers = []
    n=len(cfg)
    last_size=3
    count_m=1
    for i in range(n):
        if type(cfg[i])==int:
            layers.append(nn.Conv2d(last_size,cfg[i],(3,3),stride=1,padding=1))
            if batch_norm:
                layers.append(nn.BatchNorm2d(cfg[i]))
            layers.append(nn.ReLU())
            last_size=cfg[i]
        elif cfg[i]=='M':
            layers.append(nn.MaxPool2d((2,2),stride=2))
        else:
            print('problem !')
    return nn.Sequential(*layers)

cfg = { # M stands for max pooling 
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [ ]:
A=[64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
B=[64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
D=[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']
E=[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
net = VGG(make_layers(A),num_classes=10)

In [ ]:
def vgg_classifier(num_classes):
    classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 7 * 7, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, num_classes),
        )
    return classifier

In [ ]:
def vgg11_bn(num_classes, pretrained=False, **kwargs):
    """VGG 11-layer model (configuration "A") with batch normalization
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    if pretrained:
        kwargs['init_weights'] = False
    model = VGG(make_layers(cfg['A'], batch_norm=True), **kwargs) 
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['vgg11_bn'])) 
    model.classifier = vgg_11_classifier(num_classes)
    return model

# Train

In [12]:
def train(net, optimizer, train_loader, test_loader, criterion,  n_epoch = 5,
          train_acc_period = 100,
          test_acc_period = 5,
          cuda=True):
    loss_train = []
    loss_test = []
    total = 0
    

    for epoch in notebook.tqdm(range(n_epoch)):  # loop over the dataset multiple times
        running_loss = 0.0
        running_acc = 0.0
        for i, data in notebook.tqdm(enumerate(train_loader, 0)):
            net.train()
            # get the inputs
            inputs, labels = data
            if cuda:
                inputs = inputs.type(torch.cuda.FloatTensor)
                labels = labels.type(torch.cuda.LongTensor)
              # print(inputs.shape)
          
            # zero the parameter gradients
            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total += labels.size(0)
            
            net.eval()
            # print statistics
            '''running_loss = 0.33*loss.item()/labels.size(0) + 0.66*running_loss
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()/labels.size(0)
            running_acc = 0.3*correct + 0.66*running_acc
            if i % train_acc_period == train_acc_period-1:
                print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss))
                print('[%d, %5d] acc: %.3f' %(epoch + 1, i + 1, running_acc))
                running_loss = 0.0
                total = 0'''
            
        if epoch % test_acc_period == test_acc_period-1:
            cur_acc, cur_loss = accuracy(net, test_loader, cuda=cuda)
            print('[%d] loss: %.3f' %(epoch + 1, cur_loss))
            print('[%d] acc: %.3f' %(epoch + 1, cur_acc))
     
    print('Finished Training')

In [13]:
def accuracy(net, test_loader, cuda=True):
    net.eval()
    correct = 0
    total = 0
    loss = 0
    with torch.no_grad():
        for data in notebook.tqdm(test_loader):
            images, labels = data
            if cuda:
                images = images.type(torch.cuda.FloatTensor)
                labels = labels.type(torch.cuda.LongTensor)
            outputs = net(images)
          
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    net.train()
    print('Accuracy of the network on the test images: %d %%' % (
      100 * correct / total))
  
    return correct/total

In [14]:
#optimizer = optim.Adam(net.classifier.parameters(), lr=0.001)
optimizer = optim.SGD([
                {'params': net.features.parameters(), 'lr':1e-5},
                {'params': net.classifier.parameters(), 'lr': 1e-3}
            ], momentum=0.9)

criterion = nn.CrossEntropyLoss()

In [ ]:
#net = vgg11_bn(num_classes=1000, pretrained=True)

use_cuda = True

if use_cuda and torch.cuda.is_available():
    print("using cuda")
    net.cuda()

using cuda


In [15]:
train(net, optimizer, trainloader, testloader, criterion,  n_epoch = 10,
      train_acc_period = 50,
      test_acc_period = 100,
      cuda=False)

accuracy(net, testloader, cuda=use_cuda)

KeyboardInterrupt: ignored

In [ ]:
image = Image.open("train/0004be2cfeaba1c0361d39e2b000257b.jpg")	
image = transform(image)
print(image)

In [ ]:
train_set[1][0].shape


In [ ]:
#save the model
def save_model(net, model_save_name = 'classifier.pt'):
    path = F"{model_save_name}" 
    torch.save(net.state_dict(), path)

In [ ]:
#load the model
def load_model(model_save_name = 'classifier.pt'):
    path = F"{model_save_name}"
    model.load_state_dict(torch.load(path))

In [ ]:
save_model(net, 'vgg16 v1')

In [ ]:
accuracy(net, testloader)


Accuracy of the network on the test images: 14 %


0.14375

# A faire

- shuffle les données et baisser le lr
- augmenter epochs
- data augmentation
- modifier les batch size
- bosser sur les indicateurs de résultats
- stats sur les données initiales
- fine-tuning


"We found that finetuning a model using dropout with a small
learning rate can give much better performace than standard backpropagation finetuning."

# Historique

**vgg16 v1** (22/05 19:28)

- loss: 0.009
- acc: 0.773

With accuracy :
- trainset : 90%
- testset : 19%

Hyperparameters :
- num_epoch = 2
- batch_size = 32
- optimizer = SGD
- lr = 0.01

**vgg16 v2** (22/05 20:33)

- loss: 0.002
- acc: 0.850

With accuracy :
- trainset : 96%
- testset : 25%

Hyperparameters :
- num_epoch = 10
- batch_size = 64
- optimizer = Adam
- lr = 0.001
- shuffle = True
